In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

## Q2.2.1 ##

In [2]:
# Load Iris dataset from CSV
iris = pd.read_excel('classification iris.xlsx')
# Extract features and target
X = iris.iloc[:, :-1].values  # All columns except the last one are features
y = iris.iloc[:, -1].values   # The last column is the target
n_samples = X.shape[0]# Assume X is the feature matrix, and y is the label array
np.random.seed(1000000)# Set a random seed for reproducibility
# Generate an array of indices representing the data points
indices = np.arange(n_samples)
# Randomly shuffle the indices
np.random.shuffle(indices)
# Define the split ratio for test and train sets
test_size = 0.3
split_point = int(n_samples * test_size)
# Split the indices into test and train sets
test_indices = indices[:split_point]  # First 30% as test set indices
train_indices = indices[split_point:]  # Remaining 70% as train set indices
# get training X training y and testing X testing y
X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]
# Output the indices as comma-separated strings respectively
print("Training set indices:", '[',','.join(map(str, train_indices)), ']')
print("Test set indices:", '[',','.join(map(str, test_indices)), ']')


Training set indices: [ 100,74,126,124,4,13,36,107,71,141,65,117,42,123,82,20,106,1,62,32,21,67,121,144,97,18,86,61,59,14,46,135,22,17,28,81,31,38,116,45,73,101,92,26,7,76,43,96,87,99,138,134,84,30,47,44,147,128,51,66,15,72,114,113,127,140,55,54,98,41,33,145,37,143,130,75,119,64,79,60,16,9,132,125,142,108,52,111,63,12,6,137,120,77,89,103,139,68,110,83,91,118,53,2,56 ]
Test set indices: [ 131,109,70,149,69,34,24,78,48,104,105,11,94,19,85,8,58,133,40,10,148,95,49,50,27,112,88,80,129,115,29,25,57,146,35,5,23,102,136,0,3,93,39,90,122 ]


## Q2.2.2

In [3]:
svm = SVC(kernel="linear", C=1e5) #create svm linear model with C = 1e5
svm.fit(X_train, y_train) # fit the training data into our svm model
y_pred = svm.predict(X_test) #use svm model to predict the y value of each test point
y_pred_train = svm.predict(X_train) # the predicted value of y of the training x data

train_error = 1 - accuracy_score(y_train, y_pred_train) # calculate the training error by 1-accuracy(works the same as wrong prediction/number of data because accuracy = correct prediction/number of data)
test_error = 1 - accuracy_score(y_test, y_pred) # calculate the testing error by 1-accuracy(works the same as wrong prediction/number of data because accuracy = correct prediction/number of data)
print("Q2.2.2 Calculation using Standard SVM Model:")#title
print(f"total training error: {train_error}, total testing error: {test_error},\n")#print out total training and testing data

for i, class_name in enumerate(svm.classes_): # go over each class in order to calculate the total training and testing error
    print(f"class {class_name.split('-')[1]}:")# current class name
    train_error = 1 - accuracy_score(y_train == class_name, y_pred_train == class_name)#training error for data in each class
    test_error = 1 - accuracy_score(y_test == class_name, y_pred == class_name)#testing error for data in each class
    print(f"training error: {train_error}, testing error: {test_error},") # training and testing error in current class
    weight = svm.coef_[i] #use svm model to obtain the value of w 
    bias = svm.intercept_[i] #use sym model to obtain the value of b
    support_vectors = svm.support_ #return indexes of support vectors
    support_classes = [y_train[i] for i in support_vectors] # the class of current support vectors
    sv = [v for v, c in zip(support_vectors, support_classes) if c == class_name] # use zip to create combinations of indexes and classes and filter the support vectors that belongs to current class
    print(f"w: {weight.tolist()}, b: {bias},")# print w and b
    print(f"support vector indices: {sv},\n")# print support vector indices for current class

linear_separable = [] # create a list for all linear separable classes
for j in svm.classes_:# go over all classes
    if train_error == 0 and test_error == 0: #if both training error and testing error is 0, then add this linear separable class into the list
        linear_separable.append(j)

print('linear separable classes: ', linear_separable) # print all linear separable classes

Q2.2.2 Calculation using Standard SVM Model:
total training error: 0.0, total testing error: 0.0,

class setosa:
training error: 0.0, testing error: 0.0,
w: [-0.2619532043088393, -0.1178789419389778, 0.0, -0.20301373333935058, -0.08513479140037283], b: 14.417251080983217,
support vector indices: [54],

class versicolor:
training error: 0.0, testing error: 0.0,
w: [-0.037345639487186374, -0.0011978790024191862, -7.046347073054026e-05, -0.0032413196536048546, -0.0016206598268024275], b: 2.80318842405375,
support vector indices: [49, 58],

class virginica:
training error: 0.0, testing error: 0.0,
w: [-0.66868126606591, -0.09077285838452287, -0.023904731777931953, -0.4802939296033537, -0.2944314990436008], b: 70.80503892052188,
support vector indices: [0, 41],

linear separable classes:  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


## Q2.2.3

In [4]:
print("Q2.2.3 Calculation using SVM with Slack Variables (C=0.25 x t, where t=1,...,4):")

for t in range(1, 5):# multipliers of 0.25
    print("-------------------------------------------")
    C = 0.25 * t # apply each multiplier to achieve different values of C
    print(f"C={C},") # show current value of C
    svm = SVC(kernel="linear", C=C) #set the model linear and give our different C value into the model parameter
    svm.fit(X_train, y_train) # fit the training data into svm model
    y_pred = svm.predict(X_test) # predicted y of testing data
    y_pred_train = svm.predict(X_train) # predicted y of training data
    # Calculate errors(same as the one in 2.2.2)
    train_error = 1 - accuracy_score(y_train, y_pred_train)
    test_error = 1 - accuracy_score(y_test, y_pred)

    print(f"total training error: {train_error}, total testing error: {test_error},\n")# print total training error and testing error for each value of C

    for i, class_name in enumerate(svm.classes_): # go over each classes
        print(f"class {class_name.split('-')[1]}:") # print put current class name
        train_error = 1 - accuracy_score(y_train == class_name, y_pred_train == class_name) # training error of current class
        test_error = 1 - accuracy_score(y_test == class_name, y_pred == class_name) # testing error for current class
        print(f"training error: {train_error}, testing error: {test_error},") # print out training error and testing error of current class
        weight = svm.coef_[i] # get w from svm model
        bias = svm.intercept_[i] # get b from svm model
        support_vectors = svm.support_ #get the indexes of support vectors
        support_classes = [y_train[i] for i in support_vectors] # get the classes of support vectors
        sv = [v for v, c in zip(support_vectors, support_classes) if c == class_name] # filter the support vectors that belongs to the current class
        print(f"w: {weight.tolist()}, b: {bias},") # print out b
        print(f"support vector indices: {sv},") # print out the index of support vectors of current class
        y_class = (y_train == class_name).astype(int) # turn y into binary index, if it's in the current class then it's 1 otherwise it's -1
        y_class[y_class == 0] = -1
        x_support = X_train[sv] # get the x of current support vectors
        y_support = y_class[sv] # get the y of current support vectors
        fX = np.dot(x_support, weight.T) + bias # derive decision function
        margin_distance = y_support * fX # formulae of distance from support vectors to margin
        zeta = np.maximum(0, 1 - margin_distance) #calculate ζ
        print(f"slack variable: {zeta.tolist()},") # print out the value of ζ
        
        # add an row of space between two classes
        if i < 2:
            print()

Q2.2.3 Calculation using SVM with Slack Variables (C=0.25 x t, where t=1,...,4):
-------------------------------------------
C=0.25,
total training error: 0.0, total testing error: 0.0,

class setosa:
training error: 0.0, testing error: 0.0,
w: [-0.2619532043088393, -0.1178789419389778, 0.0, -0.20301373333935058, -0.08513479140037283], b: 14.417251080983217,
support vector indices: [54],
slack variable: [0.0],

class versicolor:
training error: 0.0, testing error: 0.0,
w: [-0.037345639487186374, -0.0011978790024191862, -7.046347073054026e-05, -0.0032413196536048546, -0.0016206598268024275], b: 2.80318842405375,
support vector indices: [49, 58],
slack variable: [1.9537970010513455, 0.1636936661831878],

class virginica:
training error: 0.0, testing error: 0.0,
w: [-0.4325633890363001, -0.05871830548185142, -0.015461966578221742, -0.31069294986733254, -0.19046196657822162], b: 45.4501382250442,
support vector indices: [0, 41],
slack variable: [2.0000264975154565, 2.0000527190363613],
---

## Q2.2.4

In [5]:
print("Q2.2.4 Calculation using SVM with Kernel Functions:")
# set parameters to 2nd-order, 3rd-order, radial basis function kernel with sigma=1 and sigmoidal kernel with sigma =1
parameters = [
    dict(kernel="poly", degree=2, C=1e5),
    dict(kernel="poly", degree=3, C=1e5),
    dict(kernel="rbf", gamma=0.5, C=1e5),
    dict(kernel="sigmoid", gamma=0.5, C=1e5)
]
# set name for each kernel
names = [
    "(a) 2nd-order Polynomial Kernel,",
    "(b) 3rd-order Polynomial Kernel,",
    "(c) Radial Basis Function Kernel with sigma=1,",
    "(d) Sigmoidal Kernel with sigma=1,"
]
# go over all the combination of all parameters and all names
for params, name in zip(parameters, names):
    print("-------------------------------------------")
    print(name)# print out the current name 
    svm = SVC(**params) # put our parameter into the svm model 
    svm.fit(X_train, y_train) # fit the training datas into the svm model 
    y_pred = svm.predict(X_test) # predicted y of testing x
    y_pred_train = svm.predict(X_train) # predicted y of training x
    # calculate training error and testing error
    train_error = 1 - accuracy_score(y_train, y_pred_train)
    test_error = 1 - accuracy_score(y_test, y_pred)
    # print out the training error and total testing error
    print(f"total training error: {train_error}, total testing error: {test_error},\n")
    # go over each class
    for i, class_name in enumerate(svm.classes_):
        print(f"class {class_name.split('-')[1]}:") # print out current class name
        train_error = 1 - accuracy_score(y_train == class_name, y_pred_train == class_name) # calculate training error 
        test_error = 1 - accuracy_score(y_test == class_name, y_pred == class_name) # calculate testing error
        print(f"training error: {train_error}, testing error: {test_error},") # print out training error and testing error
        support_vectors = svm.support_ # get index of support vectors
        support_classes = [y_train[i] for i in support_vectors] # get the class of each support vectors
        sv = [v for v, c in zip(support_vectors, support_classes) if c == class_name] # filter the support vector that belongs to the current class
        print(f"support vector indices: {sv},") # print out filtered support vector

        # add a row of space between 2 classes
        if i < 2: print()

Q2.2.4 Calculation using SVM with Kernel Functions:
-------------------------------------------
(a) 2nd-order Polynomial Kernel,
total training error: 0.0, total testing error: 0.0,

class setosa:
training error: 0.0, testing error: 0.0,
support vector indices: [54],

class versicolor:
training error: 0.0, testing error: 0.0,
support vector indices: [49, 58],

class virginica:
training error: 0.0, testing error: 0.0,
support vector indices: [0, 41],
-------------------------------------------
(b) 3rd-order Polynomial Kernel,
total training error: 0.0, total testing error: 0.0,

class setosa:
training error: 0.0, testing error: 0.0,
support vector indices: [54],

class versicolor:
training error: 0.0, testing error: 0.0,
support vector indices: [49, 58],

class virginica:
training error: 0.0, testing error: 0.0,
support vector indices: [0],
-------------------------------------------
(c) Radial Basis Function Kernel with sigma=1,
total training error: 0.0, total testing error: 0.0222222